# Finding Climate Modes with EOFs

---

## Overview
In this notebook, we will identify and plot a few different modes of climate variability with the help of an EOF package that interfaces with Xarray called [`xeofs`](https://github.com/nicrie/xeofs).

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Intro to Xarray](https://foundations.projectpythia.org/core/xarray/xarray-intro.html) | Necessary | |
| [Intro to EOFs](eof-intro) | Helpful | |

- **Time to learn**: 30 minutes

---

## Imports

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.path as mpath
from matplotlib.colors import CenteredNorm
from scipy import stats, signal
from cartopy import crs as ccrs, feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

from xeofs.xarray import EOF

## Accessing and preparing the data

We will use the [NOAA Extended Reconstructed Sea Surface Temperature version 5 (ERSSTv5)](https://www.psl.noaa.gov/data/gridded/data.noaa.oisst.v2.html) monthly gridded dataset, which is accessible using [OPeNDAP](https://www.opendap.org/). More information on [using OPeNDAP to access NOAA data can be found here](https://psl.noaa.gov/data/help/using_opendap.html).

In [ ]:
data_url = 'https://psl.noaa.gov/thredds/dodsC/Datasets/noaa.ersst.v5/sst.mnmean.nc'

In [ ]:
sst = xr.open_dataset(data_url, engine='pydap').sst
sst

Check that the data looks as expected:

In [ ]:
sst.isel(time=0).plot()

Before we modify the data, let's do an EOF analysis on the whole dataset:

In [ ]:
s_model = EOF(sst, n_modes=4, dim=['time'], weights='coslat')
s_model.solve()
s_eofs = s_model.eofs()
s_pcs = s_model.pcs()
s_expvar = s_model.explained_variance_ratio()

In [ ]:
s_eofs.plot(col='mode')

In [ ]:
s_pcs.plot(col='mode')

In [ ]:
s_expvar

EOF1 explains 83% of the variance, and the map shows interhemispheric asymmetry. The corresponding PC has a period of one year, which we can see more clearly by only plotting a few years:

In [ ]:
s_pcs.sel(mode=1, time=slice('1900', '1903')).plot(figsize=(8, 3))

This mode is showing the seasonal cycle. This is interesting, but it obfuscates other modes. If we want to study the other ways Earth's climate varies, we should remove the seasonal cycle from our data. Here we compute this (calling it the SST anomaly) by subtracting out the average of each month using Xarray's `.groupby()` method:

In [ ]:
sst_clim = sst.groupby('time.month')
ssta = sst_clim - sst_clim.mean(dim='time')

The remaining 3 EOFs show a combination of the long-term warming trend, the seasonal cycle (EOF analyses do not cleanly separate physical modes), and other internal variability. The warming trend is also interesting (see the [CMIP6 Cookbook](https://projectpythia.org/cmip6-cookbook)), but here we want to pull out some modes of internal/natural variability. We can detrend the data by removing the global average SST anomaly.

In [ ]:
def global_average(data):
    weights = np.cos(np.deg2rad(data.lat))
    data_weighted = data.weighted(weights)
    return data_weighted.mean(dim=['lat', 'lon'], skipna=True)

In [ ]:
ssta_dt = (ssta - global_average(ssta)).squeeze()

Let's find the global EOFs again but with the deseasonalized, detrended data:

In [ ]:
ds_model = EOF(ssta_dt, n_modes=4, dim=['time'], weights='coslat')
ds_model.solve()
ds_eofs = ds_model.eofs()
ds_pcs = ds_model.pcs()
ds_expvar = ds_model.explained_variance_ratio()

In [ ]:
ds_eofs.plot(col='mode')

In [ ]:
ds_pcs.plot(col='mode')

In [ ]:
ds_expvar

Now we can see some modes of variability! EOF1 looks like ENSO or IPO, and EOF2 probably picking up a pattern of the recent temperature trend where the Southern Ocean and southeastern Pacific are slightly cooling. EOF3 and EOF4 appear to be showing some decadal modes of variability (PDO and maybe AMO), among other things. There is a lot going on in each of these maps, so to get a clearer index of some modes, we can restrict our domain. 

## El Niño Southern Oscillation (ENSO)

Here we restrict our domain to the equatorial Pacific. Note that ENSO is commonly defined using an index of SST anomaly over a region of the equatorial Pacific (e.g., the [Oceanic Niño Index (ONI)](https://www.ncei.noaa.gov/access/monitoring/enso/sst)) instead of an EOF. You can [read more about ENSO here](https://www.ncei.noaa.gov/access/monitoring/enso/).

In [ ]:
ep_ssta_dt = ssta_dt.where((ssta_dt.lat < 30) & (ssta_dt.lat > -30) & (ssta_dt.lon > 120) & (ssta_dt.lon < 290), drop=True)

In [ ]:
ep_model = EOF(ep_ssta_dt, n_modes=4, dim=['time'], norm=True, weights='coslat')
ep_model.solve()
ep_eofs = ep_model.eofs()
ep_pcs = ep_model.pcs()
ep_expvar = ep_model.explained_variance_ratio()

In [ ]:
ep_eofs.plot(col='mode')

In [ ]:
ep_pcs.plot(col='mode')

In [ ]:
ep_expvar

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 2), dpi=130)
plt.fill_between(ep_pcs.time, ep_pcs.isel(mode=0).where(ep_pcs.isel(mode=0) > 0), color='r')
plt.fill_between(ep_pcs.time, ep_pcs.isel(mode=0).where(ep_pcs.isel(mode=0) < 0), color='b')
plt.ylabel('PC')
plt.xlabel('Year')
plt.xlim(ep_pcs.time.min(), ep_pcs.time.max())
plt.grid(linestyle=':')
plt.title('ENSO Index (detrended equatorial Pacific SSTA EOF1)')

Compare to the ONI:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 2), dpi=130)
plt.fill_between(ep_pcs.time, ep_pcs.isel(mode=0).where(ep_pcs.isel(mode=0) > 0), color='r')
plt.fill_between(ep_pcs.time, ep_pcs.isel(mode=0).where(ep_pcs.isel(mode=0) < 0), color='b')
plt.ylabel('PC')
plt.xlabel('Year')
plt.xlim(ep_pcs.time.sel(time='1950-01').squeeze(), ep_pcs.time.max())
plt.grid(linestyle=':')
plt.title('ENSO Index (detrended equatorial Pacific SSTA EOF1)')

<img src="images/oni.png" alt="ONI"></img>

## Pacific Decadal Oscillation (PDO)

Here we restrict our domain to the North Pacific. You can [read more about PDO here](https://www.ncei.noaa.gov/access/monitoring/pdo/).

In [ ]:
np_ssta_dt = ssta_dt.where((ssta_dt.lat < 70) & (ssta_dt.lat > 20) & (ssta_dt.lon > 120) & (ssta_dt.lon < 260), drop=True)

In [ ]:
np_model = EOF(np_ssta_dt, n_modes=4, dim=['time'], norm=True, weights='coslat')
np_model.solve()
np_eofs = np_model.eofs()
np_pcs = np_model.pcs()
np_expvar = np_model.explained_variance_ratio()

In [ ]:
np_eofs.plot(col='mode')

In [ ]:
np_pcs.plot(col='mode')

In [ ]:
np_expvar

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 2), dpi=130)
plt.fill_between(np_pcs.time, np_pcs.isel(mode=0).where(np_pcs.isel(mode=0) > 0), color='r')
plt.fill_between(np_pcs.time, np_pcs.isel(mode=0).where(np_pcs.isel(mode=0) < 0), color='b')
plt.plot(np_pcs.time, np_pcs.isel(mode=0).rolling(time=48, center=True).mean(), color='k', linewidth=2)
plt.ylabel('PC')
plt.xlabel('Year')
plt.xlim(np_pcs.time.min(), np_pcs.time.max())
plt.grid(linestyle=':')
plt.title('PDO Index (detrended North Pacific SSTA EOF1)')

---

## Summary
In this notebook, we demonstrated a basic workflow for performing an EOF analysis on gridded SST data using the `xeofs` package. We plotted the PCs associated with ENSO and PDO using deseasonalized, detrended SSTs.

### What's next?
The next section will focus on applications of EOF analysis to answer scientific questions. (Coming soon!)

## Resources and references
Huang, B., Thorne, P. S., Banzon, V., Boyer, T. P., Chepurin, G. A., Lawrimore, J. H., Menne, M. J., Smith, T. J., Vose, R. S., & Zhang, H. (2017). Extended Reconstructed Sea Surface Temperature, Version 5 (ERSSTv5): Upgrades, Validations, and Intercomparisons. *Journal of Climate*, *30*(20), 8179–8205. https://doi.org/10.1175/jcli-d-16-0836.1